In [7]:
from random import *
from tkinter import Tk, Canvas

cellsize = 25

lst_mines = []

def random_mines(p,g):
    size = len(g)
    nmines = int(size*p)
    mines = sample(g, nmines)
    return mines

def voisins(n, i, j):
    "Dans une grille n x n, indices (ligne, colonne) des voisins dans la grille d'un sommet (i,j) de la grille"
    return [(a,b) for (a, b) in [(i, j+1),(i, j-1), (i-1, j), (i+1,j), (i+1,j-1),(i+1,j+1),(i-1,j+1),(i-1,j-1)] if a in range(n) and b in range(n)]

def genere(n,cx,cy,i):
    L = []
    for x in range(0,2*i+1):
        for y in range(0,2*i+1):
            if x==0 or x==2*i or y==0 or y==2*i:
                L.append((x+cx-i,y+cy-i))
    return L

def nombre(mines,n,click):
    grid = [[0]*n for _ in range(n)]
    for (i,j) in mines:
        v = voisins(n,i,j)
        for (a,b) in v:
            if not((a,b) in mines):
                grid[a][b] += 1
                
    for (x,y) in mines:
        grid[x][y] = "X"
    
    return grid


def start(click,p):
    states = createMap(click,p)
    return states
    
def startGame(event):
    p = 0.15
    click = (event.y//cellsize,event.x//cellsize)
    if lst_mines == []:
        grid = start(click,p)
        revelle(n,grid,click)
    else:
        revelle(n,grid,click)
            
def createMap(click,p):
    grid1,p = update(n,p,click)

    #On génère une grille normalement avec toutes les mines de taille n
    grid = [(line,col) for col in range(0,n) for line in range(0,n)]
    mines = random_mines(p,grid)
    grid2 =[[0]*n for _ in range(n)]
    for (a,b) in mines:
        grid2[a][b] = "X"

    #On supperpose les deux grilles states, en faisant attention que grid1 soit prioritaire sur grid2
    map=[]
    for x in range(n):
        map.append([])
        for y in range(n):
            if (x>=click[0]-4 and x<=click[0]+4) and (y>=click[1]-4 and y<=click[1]+4):
                map[x].append(grid1[x][y])
            else:
                map[x].append(grid2[x][y])
                if grid2[x][y]=="X":
                    lst_mines.append((x,y))
    print()
    N = nombre(lst_mines,n,click)
    afficher(N)
    return N

def afficher(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            print(grid[i][j], end = " ")
        print()

#update permet de créer un grille de 4x4 au début pour créer une zones sans mines en fonction du click du joueur
def update(n,p,click):
    for i in range(1,4):
        if click[0]+i or click[0]-i or click[1]+i or click[1]-i:
            #genere permet de créer une liste qui contient 
            #les coordonnées des cases autour du click en fonction de i
            g = genere(n, click[0], click[1],i)
            mines = random_mines(p, g)
            p += 0.05
            for (a,b) in mines:
                lst_mines.append((a,b))
    
    states=[[0]*n for _ in range(n)]
    for (a,b) in lst_mines:
        states[a][b] = "X"
    return states,p

def grille(n):
    map = []
    for i in range(n):
        map.append([])
        for j in range(n):
            map[i].append(9)
    draw(map)

def draw(map):            
    for i in range(len(map)):
        for j in range(len(map[0])):
            if map[i][j]=="X":
                cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="black", outline='')
            elif map[i][j] == 9:
                cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="gray", outline='black')
            else:
                cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="white", outline='black')
                cnv.create_text(j*R+R//2, i*R+R//2, text=map[i][j], font=('courier', MAX_X//35, 'bold'))


def decouvreVoisins(p,grid,M):
    v = voisins(n,p[0],p[1])
    v.append((p[0],p[1]))
    for (i,j) in v:
        if not((i,j) in lst_mines):
            
            if grid[i][j] == 0:
                M.append((i,j))
            cnv.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="white", outline='black')
            if grid[i][j] != 9:
                cnv.create_text(j*R+R//2, i*R+R//2, text=grid[i][j], font=('courier', MAX_X//35, 'bold'))
    return M
    
def revelle(n,grid,click):
    lst_caseDecouvertes = []
    lst_caseDecouvertes = decouvreVoisins(click,grid,lst_caseDecouvertes)
    
    for k in range(3):
        for (i,j) in lst_caseDecouvertes:
            lst_caseDecouvertes = decouvreVoisins((i,j),grid,lst_caseDecouvertes)
            grid[i][j] = 9
            lst_caseDecouvertes.remove((i,j))

n=20
MAX_X,MAX_Y = n*cellsize,n*cellsize
R = MAX_X//n

root=Tk()
cnv=Canvas(root, width=MAX_X, height=MAX_Y, bg="gray")
cnv.pack()

if lst_mines == []:
    grille(n)
root.bind("<Button-1>", startGame)
    
root.mainloop()


3 X 2 0 0 0 0 1 X 1 1 X 2 X 3 3 2 3 X 2 
X X 3 1 0 0 1 2 2 2 2 2 3 3 X X X 4 X 2 
3 4 X 1 1 1 3 X 2 1 X 2 2 X 4 5 X 3 1 1 
1 X 3 2 1 X 3 X 2 1 1 2 X 4 X 4 2 2 1 1 
2 4 X 2 1 1 2 1 1 1 2 4 4 5 X X 2 2 X 2 
2 X X 2 1 1 1 1 2 3 X X X X 5 X 4 3 X 3 
2 X 4 2 3 X 3 2 X X 5 X X 5 5 X 3 X 3 X 
3 3 4 X 5 X 4 X 3 2 3 X 4 X X 2 3 2 3 1 
X X 3 X 4 X 3 1 1 0 1 1 2 2 2 1 1 X 1 0 
X 4 3 3 4 2 1 0 1 1 1 0 1 1 1 1 2 3 2 1 
X 2 1 X X 2 1 0 1 X 1 0 1 X 1 1 X 3 X 2 
3 3 2 2 3 X 2 1 2 1 1 0 2 2 3 3 4 5 X 3 
X X 2 0 1 1 2 X 1 1 1 1 1 X 2 X X X 4 X 
X X 3 1 0 0 1 2 3 3 X 1 1 1 3 3 5 3 4 X 
3 4 X 2 2 1 2 2 X X 3 2 1 1 2 X 2 X 4 3 
2 X 4 X 3 X 3 X 3 3 X 1 1 X 2 2 3 4 X X 
2 X 4 2 4 X 4 1 2 3 3 2 1 1 1 1 X 3 X X 
1 2 X 1 3 X 3 0 2 X X 2 1 1 0 2 2 4 4 4 
1 2 2 1 3 X 3 1 3 X 4 4 X 2 1 2 X 3 X X 
1 X 1 0 2 X 2 1 X 2 2 X X 2 1 X 2 3 X 3 


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\Program Files\Anaconda\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-7-e4f106a68594>", line 51, in startGame
    revelle(n,grid,click)
UnboundLocalError: local variable 'grid' referenced before assignment
